In [1]:
import time
import pandas as pd

import bernielib as bl

In [2]:
bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [3]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [4]:
ber.robotHome()
ber.pipetteHome()

# Testing how beads are mixed within 2 mL tube in sample rack

In [60]:
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 0, 800)
beads.purge()
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 0, 800)

In [61]:
beads.getVolume()

800

In [5]:
ber.pickUpNextTip()

In [63]:
ber.move(z=0)

In [64]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [65]:
ber.mixByScript(beads, script)

Executing step  0
Executing step  1
Executing step  2
Executing step  3
Executing step  4
Executing step  5
Executing step  6
Executing step  7
Executing step  8
Executing step  9
Executing step  10
Executing step  11
Executing step  12
Executing step  13
Executing step  14
Executing step  15
Executing step  16
Executing step  17
Executing step  18
Executing step  19
Executing step  20
Executing step  21
Executing step  22
Executing step  23
Executing step  24
Executing step  25
Executing step  26
Executing step  27
Executing step  28
Executing step  29
Executing step  30
Executing step  31
Executing step  32
Executing step  33
Executing step  34
Executing step  35
Executing step  36


In [66]:
ber.move(z=0)

In [12]:
samples_list = bl.createSamplesToPurifyList(ber, [50, 100, 200])

In [13]:
samples_list[0].getVolume()

50

In [15]:
ber.pipetteMove(1)

# Actual protocol

In [10]:
# Experiment parameters
number_of_samples = 3
initial_volume_list = [0, 0, 0]
beads_incubation_time = 200 # seconds
beads_pull_time = 60 # seconds

## Initializing samples and reagents

In [11]:
samples_list = bl.createSamplesToPurifyList(ber, initial_volume_list)
result_list = bl.createPurifiedSamplesList(ber, number_of_samples)

In [10]:
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 11, 800)
EtOH = bl.createSample('50ml', 'EtOH', ber.reagents_rack, 1, 0, 50000)
eluent = bl.createSample('50ml', 'eluent', ber.reagents_rack, 2, 0, 50000)
waste = bl.createSample('50ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)
EtOH80pct = bl.createSample('50ml', 'EtOH80pct', ber.reagents_rack, 2, 1, 50000)

### (technical) pre-filling the fake samples

This time, I am "purifying" the fake samples. Just filling them with water

In [12]:
def _prefillTubes(robot, samples_list, vol_list, water, safe_z=50):
    """
    This function is only used for testing. 
    It pre-fills empty tubes with water, to emulate the presence of the liquid.
    In a real protocol, samples already come with liquid.
    """
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    for sample, volume in zip(samples_list, vol_list):
        robot.transferLiquid(water, sample, volume)
    robot.move(z=safe_z)
    robot.dumpTipToWaste()

In [ ]:
_prefillTubes(ber, samples_list, [50, 100, 200], water)

## Calculating how many beads to add

In [13]:
def calcBeadsVolumeToAdd(robot, samples_list, cutoff_list):
    v_beads_list = []
    for sample, dna_size in zip(samples_list, cutoff_list):
        v_beads = robot.calcBeadsVol(sample, dna_size)
        v_beads_list.append(v_beads)
    return v_beads_list

In [14]:
v_beads_list = calcBeadsVolumeToAdd(ber, samples_list, cutoff_list=[100, 280, 900])

In [15]:
v_beads_list

[0.0, 0.0, 0.0]

## Adding beads

In [30]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [16]:
def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

In [11]:
def addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario):
    robot.move(z=0)
    robot.pickUpNextTip()
    robot.move(z=0)
    # Mixing beads before experiment
    robot.mixByScript(beads, beads_mix_scenario)
    # Transferring beads to sample
    robot.transferLiquid(beads, sample, v_beads)
    # Mixing sample with beads
    robot.mixByScript(sample, sample_mix_scenario)
    robot.move(z=0)
    robot.dumpTipToWaste()
    robot.move(z=0)

In [17]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario)
        if counter == 0:
            timestamp = time.time()
        counter += 1
    return timestamp

In [ ]:
timestamp_beads_added = addBeadsToAll(ber, samples_list, v_beads_list, beads, script, script)

In [ ]:
waitAfterTimestamp(timestamp_beads_added, beads_incubation_time)

## Removing supernatant

In [14]:
ber.moveMagnetsTowardsTube()

In [ ]:
time.sleep(beads_pull_time)

In [12]:
def removeSupernatant(robot, sample, waste):
    robot.pickUpNextTip()
    robot.move(z=0)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 0:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        robot.move(z=0)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=0)
    robot.moveToSample(sample)
    robot.movePipetteToVolume(200)
    robot.move(z=z0)
    robot.movePipetteToVolume(150)
    robot.move(z=z0-0.5)
    robot.moveAxisDelta('X', 1.5)
    robot.movePipetteToVolume(130)
    robot.moveAxisDelta('X', -1.5)
    robot.moveAxisDelta('Y', 1.5)
    robot.movePipetteToVolume(100)
    robot.moveAxisDelta('Y', -1.5)
    robot.moveAxisDelta('X', -1.5)    
    robot.movePipetteToVolume(70)
    robot.moveAxisDelta('X', 1.5)
    robot.moveAxisDelta('Y', -1.5)
    robot.movePipetteToVolume(0)
    robot.move(z=0)
    robot.dispenseLiquid(waste, 200, blow_extra=True)
    robot.move(z=0)
    robot.dumpTipToWaste()

In [ ]:
def removeSupernatantAllSamples(robot, samples_list, waste):
    for sample in samples_list:
        removeSupernatant(robot, sample, waste)

In [21]:
removeSupernatant(ber, samples_list[0], waste)

In [50]:
removeSupernatant(ber, samples_list[1], waste)

In [53]:
removeSupernatant(ber, samples_list[2], waste)

## Ethanol wash, stage 1

In [54]:
ber.pickUpNextTip()

In [55]:
ber.move(z=0)

In [57]:
ber.transferLiquid(EtOH80pct, samples_list[0], 400, touch_wall=False)
ber.transferLiquid(EtOH80pct, samples_list[1], 400, touch_wall=False)
ber.transferLiquid(EtOH80pct, samples_list[2], 400, touch_wall=False)

In [58]:
ber.move(z=0)

In [60]:
ber.dumpTipToWaste()

In [61]:
removeSupernatant(ber, samples_list[0], waste)
removeSupernatant(ber, samples_list[1], waste)
removeSupernatant(ber, samples_list[2], waste)

## Ethanol wash, stage 2

In [ ]:
ber.pickUpNextTip()
ber.move(z=0)

In [ ]:
ber.transferLiquid(EtOH80pct, samples_list[0], 400, touch_wall=False)
ber.transferLiquid(EtOH80pct, samples_list[1], 400, touch_wall=False)
ber.transferLiquid(EtOH80pct, samples_list[2], 400, touch_wall=False)

In [ ]:
ber.move(z=0)
ber.dumpTipToWaste()

In [ ]:
removeSupernatant(ber, samples_list[0], waste)
removeSupernatant(ber, samples_list[1], waste)
removeSupernatant(ber, samples_list[2], waste)

In [ ]:
# 5 minutes incubation to dry ethanol; skipped.

## Elution

In [22]:
ber.pickUpNextTip()
ber.move(z=0)

In [52]:
ber.moveMagnetsAway()

In [24]:
ber.transferLiquid(water, samples_list[0], 20)

In [13]:
def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    #z_curr = sample.calcAbsLiquidLevelFromVol(1700, added_length=robot._calcExtraLength())
    #robot.move(z=z_curr)
    #robot.moveAxisDelta('X', 4)
    #robot.movePipetteToVolume(200)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3)
    robot.movePipetteToVolume(200)
    time.sleep(delay)
    z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -1)
    z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -1)
    z_curr = sample.calcAbsLiquidLevelFromVol(10, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -1)
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.movePipetteToVolume(0)

In [46]:
ber.moveToSample(samples_list[0])

In [35]:
elutionMix(ber, samples_list[0], 20)
elutionMix(ber, samples_list[0], 20)
elutionMix(ber, samples_list[0], 20)

In [13]:
ber.move(z=50)

In [37]:
ber.moveMagnetsTowardsTube()

In [41]:
result_list = bl.createPurifiedSamplesList(ber, 1)

In [47]:
ber.movePipetteToVolume(25)
ber.movePipetteToVolume(20)
z0 = ber._getTubeZBottom(samples_list[0])
ber.move(z=z0-0.5)
ber.movePipetteToVolume(0)

In [48]:
ber.moveToSample(result_list[0])

In [49]:
ber.movePipetteToVolume(200)

In [42]:
ber.transferLiquid(samples_list[0], result_list[0], 100)

In [51]:
ber.transferLiquid(water, samples_list[0], 400, touch_wall=False)

In [14]:
ber.dumpTipToWaste()

### Adding eluent in one function

In [14]:
#TODO: make safe_z a general property of the robot
def elute(robot, sample, eluent, volume, mix_delay=0.5, mix_times=6, safe_z=50):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    robot.move(z=safe_z)
    robot.dumpTipToWaste()
    return elution_start_time

In [9]:
elute(ber, samples_list[0], water, 20)

1607418080.4168348

In [18]:
elute(ber, samples_list[2], water, 30)

1607560476.0005112

### Separate eluate from beads (one function)

In [15]:
def separateEluate(robot, eluate_tube, result_tube, pipette_delay=0.5, z_above_bottom=0.5, safe_z=50):
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    # Uptaking liquid
    v = eluate_tube.getVolume()
    robot.moveToSample(eluate_tube)
    robot.movePipetteToVolume(v+5)
    robot.movePipetteToVolume(v)
    z0 = robot._getTubeZBottom(eluate_tube)
    robot.move(z=z0-z_above_bottom)
    robot.movePipetteToVolume(0)
    time.sleep(pipette_delay)
    # Dispensing liquid
    robot.moveToSample(result_tube)
    robot.movePipetteToVolume(v+5)
    time.sleep(pipette_delay)
    robot.movePipetteToVolume(200)
    time.sleep(pipette_delay)
    robot.touchWall(result_tube)
    robot.movePipetteToVolume(0)
    robot.move(z=safe_z)
    ber.dumpTipToWaste()

In [15]:
result_list = bl.createPurifiedSamplesList(ber, 1)

In [16]:
separateEluate(ber, samples_list[0], result_list[0])

## Full elution process

In [10]:
samples_list = bl.createSamplesToPurifyList(ber, [0, 0, 0])
result_list = bl.createPurifiedSamplesList(ber, 3)

In [13]:
# Elution
# Parameters
V_water = 30
T_elute = 200 # seconds, 10 minutes
T_magnets_on = 60 # seconds

# Adding water
counter = 0
for sample in samples_list:
    ts = elute(ber, sample, water, V_water)
    if counter == 0:
        elution_start_timestamp = ts
    counter += 1

waitAfterTimestamp(elution_start_timestamp, T_elute) # elution wait

# Magnetic beads to the side of the tubes
ber.moveMagnetsTowardsTube()

time.sleep(T_magnets_on)

# Moving liquid to the resulting tubes
for sample, result in zip(samples_list, result_list):
    separateEluate(ber, sample, result)

In [17]:
a = time.time()

In [18]:
b = time.time()

In [19]:
b - a

7.7973527908325195

# Mixing procedure (continued)

In [39]:
def mix4(robot, sample, scenario, times=3, v_uptake=200, mix_delay=0.5):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_out = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    if sample._isLowVolumeUptakeNeeded(v_uptake):
        # Case when there is low liquid volume
        v_uptake_projected = sample.stype.getCloseToBottomVol()
        z_mix_projected = sample.calcAbsLiquidLevelFromVol(v_uptake_projected, added_length=robot._calcExtraLength())
        if not sample._settingPresent('tube_bottom_z'):
            # Probing the bottom
            robot.moveToSample(sample, z=z_mix_projected)
            z_tube_bottom = robot.moveDownUntilPress(step=0.2, threshold=200)
            sample.setZBottom(z_tube_bottom)
            sample.setBottomTouched()
            # Moving back up after probing
            robot.moveToSample(sample, z=z_out)
            
        z_mix = sample.getZBottom() - 1
        
        # Making sure robot will not uptake bubbles
        if v_uptake > v_in * 0.8:
            v_uptake = v_in * 0.8
            
        robot.movePipetteToVolume(v_uptake)
        
        robot.moveToSample(sample, z=z_mix)
        
        robot._pipetteUpAndDownInPlace(delay=mix_delay, times=times)
        robot.uptakeLiquid(sample, v_uptake)
        
        
        
        robot.moveToSample(sample, z=z_out)
        
        robot.pipetteMove(40)
        time.sleep(mix_delay)
        robot.pipetteMove(0)

        
        

In [42]:
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 0, 75)

In [41]:
beads.purge()

In [11]:
ber.pickUpNextTip()

In [25]:
ber.move(z=0)

In [53]:
mix4(ber, beads, scenario=mix_scenario)

## Mixing colours

In [11]:
blue = bl.createSample('eppendorf', 'blue', ber.samples_rack, 0, 0, 1000)
yellow = bl.createSample('eppendorf', 'yellow', ber.samples_rack, 1, 0, 50)

In [50]:
ber.transferLiquid(blue, yellow, 50)

In [56]:
mix4(ber, yellow, scenario=mix_scenario)

In [76]:
def mix5(robot, sample, times=3, v_uptake=200, mix_delay=0.5):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_just_above_liquid = sample.calcAbsLiquidLevelFromVol(v_in*1.2, added_length=robot._calcExtraLength())
    z_max = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    if v_uptake > v_in * 0.8:
        v_uptake = v_in * 0.8
        
    times = times - 1
    robot.uptakeLiquid(sample, v_uptake)
    z_uptake = robot.getPosition(axis='Z')
    for i in range(times):
        robot.moveToSample(sample, z=z_just_above_liquid)
        robot.dispenseLiquid(sample, v_uptake, plunger_retract=False, move_up_after=False, in_place=True)
        robot.moveToSample(sample, z=z_uptake)
        robot.uptakeLiquid(sample, v_uptake, in_place=True)
    
    robot.moveToSample(sample, z=z_max)
    robot.pipetteMove(40)
    time.sleep(mix_delay)
    robot.touchWall(sample)
    robot.pipetteMove(0)
    sample.setVolume(v_in)

In [77]:
mix5(ber, yellow, times=3)

In [10]:
yellow.getVolume()

50

In [10]:
yellow.purge()

In [16]:
yellow._settingPresent('tube_bottom_z')

False

In [19]:
ber._probeTubeZBottom(yellow)

142.43

In [24]:
ber.move(z=0)

In [23]:
ber._getTubeZBottom(yellow)

142.43

# Mixing according to the script

In [5]:
import pandas as pd

In [92]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [93]:
script

,Height,H_relative_to,Plunger_pos,dx,dy,min_vol_condition,max_vol_condition,delay,comment
0,0,top,down,0,0,0,1700,0.5,Initially lowering the plunger
1,1,bottom,up,0,0,0,250,0.5,mixing in place
2,1,bottom,down,0,0,0,250,0.5,mixing in place
3,1,bottom,up,0,0,0,250,0.5,mixing in place
4,1,bottom,down,0,0,0,250,0.5,mixing in place
5,1,bottom,up,0,0,0,250,0.5,mixing in place
6,1,bottom,down,0,0,0,250,0.5,mixing in place
7,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
8,5,surface,down,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"
9,1,bottom,up,0,0,0,1700,0.5,"uptaking from bottom, releasing to the top"


In [8]:
def mixByScript(robot, sample, script, vol_uptake_fraction=0.8):
    v_in = sample.getVolume()
    v_max = sample.stype.getMaxVolume()
    z_liquid = sample.calcAbsLiquidLevelFromVol(v_in, added_length=robot._calcExtraLength())
    z_max = sample.calcAbsLiquidLevelFromVol(v_max, added_length=robot._calcExtraLength())
    
    number_of_steps = script.shape[0]
    for step in range(number_of_steps):
        
        print ("Executing step ", step)
        
        step_params = script.loc[step]
        # Parameters
        h = step_params.Height
        h_relative_to = step_params.H_relative_to
        plunger_position = step_params.Plunger_pos
        dx = step_params.dx
        dy = step_params.dy
        condition_v_min = step_params.min_vol_condition
        condition_v_max = step_params.max_vol_condition
        d = step_params.delay
        
        # Do I need to skip this step entirely?
        if v_in > condition_v_min and v_in < condition_v_max:
            # perform step
        
            # Figuring out absolute Z of the operation. If necessary, will probe the tube bottom.
            if h_relative_to == 'top':
                z = z_max + h # Getting deeper into the tube compared to the tube top. Positive -> deeper
            elif h_relative_to == 'bottom':
                z_bottom = robot._getTubeZBottom(sample)
                z = z_bottom - h # Above the absolute Z of the tube.
            elif h_relative_to == 'surface':
                z = z_liquid - h # Above the surface level
            else:
                print ("Please provide valid H_relative_to value. Valid values are top, bottom, surface.")
            
            # Identifying uptake volume
            v_uptake = v_in * vol_uptake_fraction
            if v_uptake > 200:
                v_uptake = 200
            
            if plunger_position == 'up':
                p = 0
            elif plunger_position == 'down':
                p = robot.calcPipettePositionFromVolume(v_uptake)
            else:
                print ("Please provide valid Plunger_pos value. Valid values are up and down.")
        
            # Moving to the position for the operation
            robot.moveToSample(sample, z=z, z_hop=0)
            robot.moveAxisDelta('X', dx)
            robot.moveAxisDelta('Y', dy)
        
            # Performing pipette operation (actual mixing)
            robot.pipetteMove(p)
            time.sleep(d)
            
            # Moving back x and y:
            robot.moveAxisDelta('X', -dx)
            robot.moveAxisDelta('Y', -dy)
            
            # Now getting to the next step.
            
        
        
        #print (h, h_relative_to, plunger_position, dx, dy, v_min, v_max, d)
    
    #Purging at the end of the protocol
    robot.moveToSample(sample, z=z_max, z_hop=0)
    robot.pipetteMove(40)
    time.sleep(d)
    robot.touchWall(sample)
    robot.pipetteMove(0)

In [53]:
yellow = bl.createSample('eppendorf', 'yellow', ber.samples_rack, 1, 0, 600)

In [10]:
ber.pickUpNextTip()

In [52]:
ber.move(z=0)

In [126]:
mixByScript(ber, yellow, script)

Executing step  0
Executing step  1
Executing step  2
Executing step  3
Executing step  4
Executing step  5
Executing step  6
Executing step  7
Executing step  8
Executing step  9
Executing step  10
Executing step  11
Executing step  12
Executing step  13
Executing step  14
Executing step  15
Executing step  16
Executing step  17
Executing step  18
Executing step  19
Executing step  20
Executing step  21
Executing step  22
Executing step  23
Executing step  24
Executing step  25
Executing step  26
Executing step  27
Executing step  28
Executing step  29
Executing step  30
Executing step  31
Executing step  32
Executing step  33
Executing step  34
Executing step  35
Executing step  36


In [127]:
ber.move(z=0)

In [66]:
ber.moveMagnetsTowardsTube()

In [67]:
ber.moveMagnetsAway()

# Settings to be entered just once

In [5]:
ber.setBeadsVolumeCoef(a=4.99325349e-01, b=-9.91043764e+00, c=2.57585836e+04)

In [6]:
ber.getBeadsVolumeCoef()

(0.499325349, -9.91043764, 25758.5836)

In [7]:
test_type = bl.sample_type('eppendorf')

In [8]:
test_sample = bl.sample('test_sample', test_type)

In [14]:
test_sample.setVolume(50)

In [16]:
ber.calcBeadsVol(test_sample, 1000)

25.758674747999997

In [43]:
ber.pipetteServoUp()

In [42]:
ber.pipetteServoDown()

In [51]:
ber.calibrateRack(rack='tips')

(76.80000000000001, 150.165, 168.9)

In [28]:
ber.calibrateRack(rack='samples')

(175.5, 128.20499999999998, 155.3)

In [5]:
ber.calibrateRack(rack='waste')

Error: setting  calibration_style  is not specified.
Use _setSetting('setting_name', value) to specify it.
Alternatively, do self.data['setting_name']=value to set it temporary.
Provided calibration style is not supported.
Please use 'outer', 'inner' or 'well' styles.
Current provided style is:  None
Error: setting  calibration_Z  is not specified.
Use _setSetting('setting_name', value) to specify it.
Alternatively, do self.data['setting_name']=value to set it temporary.


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [26]:
ber.move(z=180, speed_z=4000)

In [29]:
ber.move(z=0)

In [111]:
ber.writeAndWaitCartesian("M503")

'echo:  G21    ; Units in mm\n\necho:Filament settings: Disabled\necho:  M200 D3.00\necho:  M200 D0\necho:Steps per unit:\necho:  M92 X10.00 Y10.00 Z100.00 E500.00\necho:Maximum feedrates (units/s):\necho:  M203 X2000.00 Y2000.00 Z4000.00 E25.00\necho:Maximum Acceleration (units/s2):\necho:  M201 X10000 Y10000 Z100 E10000\necho:Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>\necho:  M204 P250.00 R10000.00 T250.00\necho:Advanced: S<min_feedrate> T<min_travel_feedrate> B<min_segment_time_us> X<max_xy_jerk> Z<max_z_jerk> E<max_e_jerk>\necho:  M205 S0.00 T0.00 B20000 X2.00 Y2.00 Z1.00 E5.00\necho:Home offset:\necho:  M206 X0.00 Y0.00 Z0.00\necho:PID settings:\necho:  M301 P22.20 I1.08 D114.00\necho:Z-Probe Offset (mm):\necho:  M851 Z0.00\nok\n'

In [128]:
ber.writeAndWaitCartesian("M201 Z200")

'ok\n'

In [6]:
ber.move(z=180)